In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from modules import RCGAN
import random
import hparams as hp
import numpy as np
import pandas as pd
from utils import *
from torch.utils.data import DataLoader
from tqdm import tqdm


pid_list = os.listdir("./Dataset/physionet.org/files/eicu-crd/2.0/preprocessed/sequences")
random.seed(1234)
random.shuffle(pid_list)

seq_list = []
for pid in tqdm(pid_list[:int(0.8*len(pid_list))]):
    seq = np.load(f"./Dataset/physionet.org/files/eicu-crd/2.0/preprocessed/sequences/{pid}")
    seq_list.append(seq)

mu, std = np.concatenate(seq_list, axis=0).mean(axis=0), np.concatenate(seq_list, axis=0).std(axis=0)

model = RCGAN(hp).cuda()
checkpoint_dict = torch.load("./training_log/eICU/RCGAN_checkpoint_440000.pt", map_location='cpu')
model.load_state_dict(checkpoint_dict['state_dict'])
model.eval()

for pid in tqdm(pid_list):
    label = np.load(f"./Dataset/physionet.org/files/eicu-crd/2.0/preprocessed/labels/{pid}")
    preds = model(torch.LongTensor(label).unsqueeze(0).cuda())
    
    seq = std*preds[0].detach().cpu().numpy()+mu
    np.save(f"./synthetic_dataset/sequences/{pid}", seq)
    np.save(f"./synthetic_dataset/labels/{pid}", label)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 71739/71739 [06:39<00:00, 179.79it/s]
